## Libraries

In [1]:
#main
import pandas as pd
import numpy as np
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path
import random

#helpers
import uuid
import pandas as pd
import numpy as np


## Load Documents


In [2]:
data_dir = "VU"
inputdirectory = Path(f"./inp/{data_dir}")

## This is where the output csv files will be written
out_dir = data_dir
outputdirectory = Path(f"./data_output/{out_dir}") 

# Ensure input directory exists
if not os.path.isdir(inputdirectory):
    print(f"Input directory {inputdirectory} doesn't exist!")
else:
    print(f"Input directory {inputdirectory} exists.")

Input directory inp\VU exists.


In [3]:
## Dir PDF Loader
loader = PyPDFDirectoryLoader(inputdirectory)
## File Loader
# loader = PyPDFLoader("./data/MedicalDocuments/orf-path_health-n1.pdf")
# loader = DirectoryLoader(inputdirectory, show_progress=True)
documents = loader.load()


splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=150,
    length_function=len,
    is_separator_regex=False,
)

pages = splitter.split_documents(documents)
print("Number of chunks = ", len(pages))
# print(pages[0].page_content)

Number of chunks =  8


## Document to dataframe

In [4]:
def documents2Dataframe(documents) -> pd.DataFrame:
    rows = []
    for chunk in documents:
        row = {
            "text": chunk.page_content,
            **chunk.metadata,
            "chunk_id": uuid.uuid4().hex,
        }
        rows = rows + [row]

    df = pd.DataFrame(rows)
    return df

def extractConcepts(dataframe: pd.DataFrame, model=None) -> list:
    # dataframe.reset_index(inplace=True)
    results = dataframe.apply(
        lambda row: graphPrompt(row.text, {"chunk_id": row.chunk_id}, model), axis=1
    )
    # invalid json results in NaN
    results = results.dropna()
    results = results.reset_index(drop=True)

    ## Flatten the list of lists to one single list of entities.
    concept_list = np.concatenate(results).ravel().tolist()
    return concept_list


def cleanConcepts2df(nodes_list) -> pd.DataFrame:
    ## Remove all NaN entities
    graph_dataframe = pd.DataFrame(nodes_list).replace(" ", np.nan)
    graph_dataframe = graph_dataframe.dropna(subset=["node_1", "node_2"])
    graph_dataframe["node_1"] = graph_dataframe["node_1"].apply(lambda x: x.lower())
    graph_dataframe["node_2"] = graph_dataframe["node_2"].apply(lambda x: x.lower())

    return graph_dataframe


In [5]:
df = documents2Dataframe(pages)
print(df.shape)

pd.set_option("display.max_rows", None)  # Show all rows
pd.set_option("display.max_columns", None)  # Show all columns
pd.set_option("display.width", None)  # No limit on width
pd.set_option("display.max_colwidth", None)  # No limit on column width

df.head()

# print(df.apply(lambda row: print(row.text), axis=1))


(8, 10)


,text,producer,creator,creationdate,moddate,source,total_pages,page,page_label,chunk_id
0,1,Microsoft® Word 2021,Microsoft® Word 2021,2025-03-01T15:26:23+05:30,2025-03-01T15:26:23+05:30,inp\VU\Researchh.pdf,6,0,1,611810229e2848a79dd6c9c931c33d9a
1,2 \nContents \n \n1 Introduction 3 \n2 Brief History of Virtual Unwrapping Techniques 4 \n3 How It Works 5 \n4 Conclusion 6 \n5 References 6,Microsoft® Word 2021,Microsoft® Word 2021,2025-03-01T15:26:23+05:30,2025-03-01T15:26:23+05:30,inp\VU\Researchh.pdf,6,1,2,383af29026c14c05aa5862a3a8051ce7
2,"3 \n \n1 Introduction \nIn fact, this digital procedure is called ”virtual unwrapping,” which is a procedure by \nwhich tangible contents hidden within sealed or delicate items, are made accessible with- \nout having to break them open. Among its prime applications are medical imaging, \nconservation of cultural heritage, and archeology. \nVirtual unwrapping has thus now become a view -access technology of studying the \ninnards of fragile, ancient items non -invasively in a world of breakthrough combination \nof cutting-edge imaging with computer approaches. Forensic science, artistic restoration, \nand archaeology have embraced this approach that was originally developed to tackle \nthe issues in conserving delicate, ancient writings and artifacts, such as the infamous \nHerculaneum scrolls. Virtual unwrapping thus keeps priceless cultural material away \nfrom the dangers of physical handling and, at the same time, reveals very small details \nthat might otherwise remain hidden. \nWith the recent advances in wildlife detection systems using synchrotron radiation, \nmicro-CT, and X -ray computed tomography (CT), we have truly crossed a new frontier \nin terms of what we are able to achieve with internal structure imaging. Hence, coupled \nwith sophisticated digital reconstruction algorithms, these types of imaging advancements \nempower researchers to virtually ”unwrap” objects, allowing layers of text, and images,",Microsoft® Word 2021,Microsoft® Word 2021,2025-03-01T15:26:23+05:30,2025-03-01T15:26:23+05:30,inp\VU\Researchh.pdf,6,2,3,03273abb977c4094b1c423e7f52df363
3,"empower researchers to virtually ”unwrap” objects, allowing layers of text, and images, \nand actual material composition to escape much like one would have done by unwrapping \nwithout damaging any original artifact",Microsoft® Word 2021,Microsoft® Word 2021,2025-03-01T15:26:23+05:30,2025-03-01T15:26:23+05:30,inp\VU\Researchh.pdf,6,2,3,91eac16a286f44079c01c2d5e490fa79
4,"4 \n2 How It Works \nVirtual Unwrapping typically involves the following steps: \n \n1. Scanning \n• CT imaging is a process whereby an X-ray source emits X-rays through a thin beam \nalmost perpendicular to the object and very rapidly revolves about the subject \nto create signals that were processed by the machine into images or slices of that \nsubject. These slices will then be stacked to produce a 3D image.-Different state-of- \nthe-art high-resolution imaging techniques like X -ray computed tomography (CT), \nmicro-CT, or Synchrotron Radiation have been used to give internal details of the \nobject. \n \n2. Segmentation & Layer Detection \n• Advanced algorithms are responsible for identifying the different layers within the \nscanned data and assist the researchers in isolating the text, drawings, or any other \nhidden elements. The actual 3D data is then further sub -segmented into distinct \nlayers or regions, once again aided by advanced computational techniques. The \ntechnique makes use of subtle texture and density variations to separate effectively \nwriting planes or other features from all objects. The whole decision-making at this \nstep is vital since this is what every further investigation will ultimately depend \nupon. \n \n3. Texture Mapping \n• It serves to accentuate any text or images that are faintly discernible. Thereafter, \nfollowing segmentation, algorithms correlate changes in intensity to ascertain the",Microsoft® Word 2021,Mi

## Extract Concepts from the Dataframe

!!The ollama.client doesnt work because client is a file in the main file!!


In [ ]:
import sys
from yachalk import chalk
sys.path.append("..")

import json
import ollama


def graphPrompt(input: str, metadata={}, model="mistral:latest"):
    if model == None:
        model = "mistral:latest"



    SYS_PROMPT = (
        "You are a network graph maker who extracts terms and their relations from a given context. "
        "You are provided with a context chunk (delimited by ```) Your task is to extract the ontology "
        "of terms mentioned in the given context. These terms should represent the key concepts as per the context. \n"
        "Thought 1: While traversing through each sentence, Think about the key terms mentioned in it.\n"
            "\tTerms may include object, entity, location, organization, person, \n"
            "\tcondition, acronym, documents, service, concept, etc.\n"
            "\tTerms should be as atomistic as possible\n\n"
        "Thought 2: Think about how these terms can have one on one relation with other terms.\n"
            "\tTerms that are mentioned in the same sentence or the same paragraph are typically related to each other.\n"
            "\tTerms can be related to many other terms\n\n"
        "Thought 3: Find out the relation between each such related pair of terms. \n\n"
        "Format your output as a list of json. Each element of the list contains a pair of terms"
        "and the relation between them, like the follwing: \n"
        "[\n"
        "   {\n"
        '       "node_1": "A concept from extracted ontology",\n'
        '       "node_2": "A related concept from extracted ontology",\n'
        '       "edge": "relationship between the two concepts, node_1 and node_2 in one or two sentences"\n'
        "   }, {...}\n"
        "]"
    )

    USER_PROMPT = f"context: ```{input}``` \n\n output: "
    response = ollama.chat(
        model="mistral:latest",
        messages=[
            {"role": "system", "content": SYS_PROMPT},  # Add system prompt here
            {"role": "user", "content": USER_PROMPT}
        ],
        options={"use_gpu": True}
    )

    try:

        result = json.loads(response['message']['content'])  
        result = [dict(item, **metadata) for item in result]
    except Exception as e:
        print("\n\nERROR ### Here is the buggy response: ", response, "\n\n")
        print("Exception:", str(e))
        result = None

    return result


In [7]:
## To regenerate the graph with LLM, set this to True
regenerate = True

if regenerate:
    concepts_list = extractConcepts(df, model='mistral:latest')
    dfg1 = cleanConcepts2df(concepts_list)
    if not os.path.exists(outputdirectory):
        os.makedirs(outputdirectory)
    
    dfg1.to_csv(outputdirectory/"graph.csv", sep="|", index=False)
    df.to_csv(outputdirectory/"chunks.csv", sep="|", index=False)
else:
    dfg1 = pd.read_csv(outputdirectory/"graph.csv", sep="|")

dfg1.replace("", np.nan, inplace=True)
dfg1.dropna(subset=["node_1", "node_2", 'edge'], inplace=True)
dfg1['count'] = 10
## Increasing the weight of the relation to 4. 
## We will assign the weight of 1 when later the contextual proximity will be calculated.  
pd.set_option("display.max_rows", None)  # Show all rows
pd.set_option("display.max_columns", None)  # Show all columns
pd.set_option("display.width", None)  # No limit on width
pd.set_option("display.max_colwidth", None)  # No limit on column width
print(dfg1.shape)
dfg1.head()

(65, 5)


,node_1,node_2,edge,chunk_id,count
0,traversing,through,"traversing is the action of moving through a space or area, and 'through' indicates the direction or path taken during this movement",611810229e2848a79dd6c9c931c33d9a,10
1,each sentence,think about,"'Each sentence' refers to the text being considered for analysis, and 'Think about' is a command or instruction given to the system to focus on certain aspects while analyzing the sentence.",611810229e2848a79dd6c9c931c33d9a,10
2,key terms,mentioned,"'Key terms' are important concepts or ideas within the text, and 'mentioned' signifies that these key terms have been explicitly stated in the context.",611810229e2848a79dd6c9c931c33d9a,10
3,object,entity,"In this context, both 'object' and 'entity' refer to something that can be recognized or identified, but 'object' typically refers to a tangible thing while 'entity' can encompass both tangible objects and abstract concepts.",611810229e2848a79dd6c9c931c33d9a,10
4,location,organization,"'Location' often refers to a place or area, whereas 'organization' typically denotes a group of individuals or things that work together towards a common goal. However, organizations can also have locations where they operate.",611810229e2848a79dd6c9c931c33d9a,10


In [8]:
def contextual_proximity(df: pd.DataFrame) -> pd.DataFrame:
    ## Melt the dataframe into a list of nodes
    dfg_long = pd.melt(
        df, id_vars=["chunk_id"], value_vars=["node_1", "node_2"], value_name="node"
    )
    dfg_long.drop(columns=["variable"], inplace=True)
    # Self join with chunk id as the key will create a link between terms occuring in the same text chunk.
    dfg_wide = pd.merge(dfg_long, dfg_long, on="chunk_id", suffixes=("_1", "_2"))
    # drop self loops
    self_loops_drop = dfg_wide[dfg_wide["node_1"] == dfg_wide["node_2"]].index
    dfg2 = dfg_wide.drop(index=self_loops_drop).reset_index(drop=True)
    ## Group and count edges.
    dfg2 = (
        dfg2.groupby(["node_1", "node_2"])
        .agg({"chunk_id": [",".join, "count"]})
        .reset_index()
    )
    dfg2.columns = ["node_1", "node_2", "chunk_id", "count"]
    dfg2.replace("", np.nan, inplace=True)
    dfg2.dropna(subset=["node_1", "node_2"], inplace=True)
    # Drop edges with 1 count
    dfg2 = dfg2[dfg2["count"] != 1]
    dfg2["edge"] = "contextual proximity"
    return dfg2


dfg2 = contextual_proximity(dfg1)
dfg2.tail()

,node_1,node_2,chunk_id,count,edge
1257,x-ray source,slices,"e3ae5e15c22547239fe3ccd407c7b423,e3ae5e15c22547239fe3ccd407c7b423",2,contextual proximity
1258,x-ray source,subject,"e3ae5e15c22547239fe3ccd407c7b423,e3ae5e15c22547239fe3ccd407c7b423",2,contextual proximity
1259,x-ray source,subtle texture and density variations,"e3ae5e15c22547239fe3ccd407c7b423,e3ae5e15c22547239fe3ccd407c7b423",2,contextual proximity
1260,x-ray source,synchrotron radiation,"e3ae5e15c22547239fe3ccd407c7b423,e3ae5e15c22547239fe3ccd407c7b423",2,contextual proximity
1261,x-ray source,text or images,"e3ae5e15c22547239fe3ccd407c7b423,e3ae5e15c22547239fe3ccd407c7b423",2,contextual proximity


## Merge both dataframes


In [9]:
dfg = pd.concat([dfg1, dfg2], axis=0)
dfg = (
    dfg.groupby(["node_1", "node_2"])
    .agg({"chunk_id": ",".join, "edge": ','.join, 'count': 'sum'})
    .reset_index()
)
dfg
# dfg = pd.concat([dfg1, dfg2], axis=0, ignore_index=True)

# dfg = dfg.groupby(["node_1", "node_2"], as_index=False).agg({
#     "chunk_id": ",".join, 
#     "edge": lambda x: ','.join(set(x)),  # Avoid duplicate edges in text
#     "count": "sum"  # Ensure the count is properly accumulated
# })
# dfg

,node_1,node_2,chunk_id,edge,count
0,3d data,advanced algorithms,"e3ae5e15c22547239fe3ccd407c7b423,e3ae5e15c22547239fe3ccd407c7b423",contextual proximity,2
1,3d data,ct imaging,"e3ae5e15c22547239fe3ccd407c7b423,e3ae5e15c22547239fe3ccd407c7b423,e3ae5e15c22547239fe3ccd407c7b423",contextual proximity,3
2,3d data,distinct layers or regions,e3ae5e15c22547239fe3ccd407c7b423,The 3D data is sub-segmented into distinct layers or regions.,10
3,3d data,high-resolution imaging techniques,"e3ae5e15c22547239fe3ccd407c7b423,e3ae5e15c22547239fe3ccd407c7b423,e3ae5e15c22547239fe3ccd407c7b423",contextual proximity,3
4,3d data,x-ray source,"e3ae5e15c22547239fe3ccd407c7b423,e3ae5e15c22547239fe3ccd407c7b423",contextual proximity,2
5,3d image,advanced algorithms,"e3ae5e15c22547239fe3ccd407c7b423,e3ae5e15c22547239fe3ccd407c7b423",contextual proximity,2
6,3d image,ct imaging,"e3ae5e15c22547239fe3ccd407c7b423,e3ae5e15c22547239fe3ccd407c7b423,e3ae5e15c22547239fe3ccd407c7b423",contextual proximity,3
7,3d image,high-resolution imaging techniques,"e3ae5e15c22547239fe3ccd407c7b423,e3ae5e15c22547239fe3ccd407c7b423,e3ae5e15c22547239fe3ccd407c7b423",contextual proximity,3
8,3d image,x-ray source,"e3ae5e15c22547239fe3ccd407c7b423,e3ae5e15c22547239fe3ccd407c7b423",contextual proximity,2
9,5 references,virtual unwrapping techniques,"383af29026c14c05aa5862a3a8051ce7,383af29026c14c05aa5862a3a8051ce7,383af29026c14c05aa5862a3a8051ce7,383af29026c14c05aa5862a3a8051ce7",contextual proximity,4


In [10]:
nodes = pd.concat([dfg['node_1'], dfg['node_2']], axis=0).unique()
nodes.shape

(90,)

In [11]:
import networkx as nx
G = nx.Graph()

## Add nodes to the graph
for node in nodes:
    G.add_node(
        str(node)
    )

## Add edges to the graph
for index, row in dfg.iterrows():
    G.add_edge(
        str(row["node_1"]),
        str(row["node_2"]),
        title=row["edge"],
        weight=row['count']/4
    )

In [12]:
communities_generator = nx.community.girvan_newman(G)
top_level_communities = next(communities_generator)
next_level_communities = next(communities_generator)
communities = sorted(map(sorted, next_level_communities))
print("Number of Communities = ", len(communities))
print(communities)

Number of Communities =  11
[['3d data', '3d image', 'advanced algorithms', 'advanced computational techniques', 'ct', 'ct imaging', 'distinct layers or regions', 'faintly discernible', 'high-resolution imaging techniques', 'identifying the different layers within the scanned data', 'micro-ct', 'processing machine', 'researchers', 'revolves about the subject', 'slices', 'subject', 'subtle texture and density variations', 'synchrotron radiation', 'text or images', 'x-ray source'], ['5 references', 'brief history of virtual unwrapping techniques', 'conclusion', 'how it works', 'introduction', 'references', 'virtual unwrapping techniques'], ['acronym', 'documents'], ['advances in computing algorithms', 'advancing scientific research', 'artificial intelligence', 'artistic restoration', 'challenges', 'computational complexity', 'conservation of cultural heritage', 'conserving cultural artifacts', 'deep learning algorithms and imaging technologies', 'digital preservation', 'digital reconstru

In [13]:
import seaborn as sns
palette = "hls"

## Now add these colors to communities and make another dataframe
def colors2Community(communities) -> pd.DataFrame:
    ## Define a color palette
    p = sns.color_palette(palette, len(communities)).as_hex()
    random.shuffle(p)
    rows = []
    group = 0
    for community in communities:
        color = p.pop()
        group += 1
        for node in community:
            rows += [{"node": node, "color": color, "group": group}]
    df_colors = pd.DataFrame(rows)
    return df_colors


colors = colors2Community(communities)
colors

,node,color,group
0,3d data,#5767db,1
1,3d image,#5767db,1
2,advanced algorithms,#5767db,1
3,advanced computational techniques,#5767db,1
4,ct,#5767db,1
5,ct imaging,#5767db,1
6,distinct layers or regions,#5767db,1
7,faintly discernible,#5767db,1
8,high-resolution imaging techniques,#5767db,1
9,identifying the different layers within the scanned data,#5767db,1


In [14]:
for index, row in colors.iterrows():
    G.nodes[row['node']]['group'] = row['group']
    G.nodes[row['node']]['color'] = row['color']
    G.nodes[row['node']]['size'] = G.degree[row['node']]

In [15]:
# !pip install pyvis

from pyvis.network import Network

graph_output_directory = "./docs/index.html"

net = Network(
    notebook=False,
    # bgcolor="#1a1a1a",
    cdn_resources="remote",
    height="900px",
    width="100%",
    select_menu=True,
    # font_color="#cccccc",
    filter_menu=False,
)

net.from_nx(G)
# net.repulsion(node_distance=150, spring_length=400)
net.force_atlas_2based(central_gravity=0.015, gravity=-31)
# net.barnes_hut(gravity=-18100, central_gravity=5.05, spring_length=380)
net.show_buttons(filter_=["physics"])

net.show(graph_output_directory, notebook=False)

./docs/index.html
